In [ ]:
# Installing Dependency
!pip install snowflake-snowpark-python
!pip install snowflake-connector-python
!pip install ipykernel
!pip install pyarrow == 10.0.1
!pip install numpy
!pip install scikit-learn
!pip install pandas
!pip install joblib
!pip install cachetools
!pip install xgboost

In [1]:
import os
from snowflake.connector import connect
from snowflake.snowpark import functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark import version as v

connection_parameters = {
   
  "account": "",
  "user": "",
  "password": "",
  "warehouse": "",
  "database": "",
  "schema": ""
}

session = Session.builder.configs(connection_parameters).create()

In [2]:
#creating a database 'snowflake_sample_data' from sfc_samples.sample_data. Where in will have DDL amd DML rights
session.sql('''create database if not exists snowflake_sample_data from share sfc_samples.sample_data''').collect()

[Row(status='SNOWFLAKE_SAMPLE_DATA already exists, statement succeeded.')]

In [3]:
session.sql('CREATE DATABASE IF NOT EXISTS tpcds_xgboost').collect()

[Row(status='TPCDS_XGBOOST already exists, statement succeeded.')]

In [4]:
#Creating tpcds_xgboost Database
session.sql('CREATE DATABASE IF NOT EXISTS tpcds_xgboost').collect()

#Creating schema tpcds_xgboost.demo
session.sql('CREATE SCHEMA IF NOT EXISTS tpcds_xgboost.demo').collect()

#creating two different warehouses for different workload
session.sql("create or replace warehouse FE_AND_INFERENCE_WH with warehouse_size='3X-LARGE'").collect()
session.sql("create or replace warehouse snowpark_opt_wh with warehouse_size = 'MEDIUM' warehouse_type = 'SNOWPARK-OPTIMIZED'").collect()
session.sql("alter warehouse snowpark_opt_wh set max_concurrency_level = 1").collect()

#setting current warehouse to be FE_AND_INFERENCE_WH
session.use_warehouse('FE_AND_INFERENCE_WH')

In [5]:
#Snowflake provide different datasets size to work with. Will provide flexibility to choose the size 
TPCDS_SIZE_PARAM = 100
SNOWFLAKE_SAMPLE_DB = 'SNOWFLAKE_SAMPLE_DATA' 

if TPCDS_SIZE_PARAM == 100: 
    TPCDS_SCHEMA = 'TPCDS_SF100TCL'
elif TPCDS_SIZE_PARAM == 10:
    TPCDS_SCHEMA = 'TPCDS_SF10TCL'
else:
    raise ValueError("Invalid TPCDS_SIZE_PARAM selection")


In [6]:
#Storing table content as spark dataframe
store_sales = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.store_sales')
catalog_sales = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.catalog_sales') 
web_sales = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.web_sales') 
date = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.date_dim')
dim_stores = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.store')
customer = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.customer')
address = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.customer_address')
demo = session.table(f'{SNOWFLAKE_SAMPLE_DB}.{TPCDS_SCHEMA}.customer_demographics')

# Data Transformation

In [7]:
#Aggregating data
store_sales_agged = store_sales.group_by('ss_customer_sk').agg(F.sum('ss_sales_price').as_('total_sales'))
web_sales_agged = web_sales.group_by('ws_bill_customer_sk').agg(F.sum('ws_sales_price').as_('total_sales'))
catalog_sales_agged = catalog_sales.group_by('cs_bill_customer_sk').agg(F.sum('cs_sales_price').as_('total_sales'))

#Renaming the columns
store_sales_agged = store_sales_agged.rename('ss_customer_sk', 'customer_sk')
web_sales_agged = web_sales_agged.rename('ws_bill_customer_sk', 'customer_sk')
catalog_sales_agged = catalog_sales_agged.rename('cs_bill_customer_sk', 'customer_sk')

In [8]:
#Stacking all the three aggregated tables to have one table
total_sales = store_sales_agged.union_all(web_sales_agged)
total_sales = total_sales.union_all(catalog_sales_agged)

In [9]:
#Aggregating to have unique customer_sk
total_sales = total_sales.group_by('customer_sk').agg(F.sum('total_sales').as_('total_sales'))


In [10]:
#Select set of columns from customer table
customer = customer.select('c_customer_sk','c_current_hdemo_sk', 'c_current_addr_sk', 'c_customer_id', 'c_birth_year')

In [11]:
#Joining address and demograhic table with cutomer table to create a custom customer table
customer = customer.join(address.select('ca_address_sk', 'ca_zip'), customer['c_current_addr_sk'] == address['ca_address_sk'] )
customer = customer.join(demo.select('cd_demo_sk', 'cd_gender', 'cd_marital_status', 'cd_credit_rating', 'cd_education_status', 'cd_dep_count'),
                                customer['c_current_hdemo_sk'] == demo['cd_demo_sk'] )
customer = customer.rename('c_customer_sk', 'customer_sk')
customer.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_SK"  |"C_CURRENT_HDEMO_SK"  |"C_CURRENT_ADDR_SK"  |"C_CUSTOMER_ID"   |"C_BIRTH_YEAR"  |"CA_ADDRESS_SK"  |"CA_ZIP"  |"CD_DEMO_SK"  |"CD_GENDER"  |"CD_MARITAL_STATUS"  |"CD_CREDIT_RATING"  |"CD_EDUCATION_STATUS"  |"CD_DEP_COUNT"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2435782        |5105                  |22514086             |AAAAAAAAGMKCFCAA  |1954            |22514086         |NULL      |5105          |M            |D                    |Unknown             |Unknown                |0               |
|2436098        |6723               

In [12]:
#Finally joining aggregated sales and customer table
final_df = total_sales.join(customer, on='customer_sk')

In [13]:
#Storing final table in Snowflake
session.use_database('tpcds_xgboost')
session.use_schema('demo')
final_df.write.mode('overwrite').save_as_table('feature_store')

In [14]:
session.add_packages('snowflake-snowpark-python', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'cachetools', 'xgboost', 'joblib')

The version of package snowflake-snowpark-python in the local environment is 1.2.0, which does not fit the criteria for the requirement snowflake-snowpark-python. Your UDF might not work when the package version is different between the server and your local environment
The version of package scikit-learn in the local environment is 1.2.2, which does not fit the criteria for the requirement scikit-learn. Your UDF might not work when the package version is different between the server and your local environment
The version of package pandas in the local environment is 1.5.3, which does not fit the criteria for the requirement pandas. Your UDF might not work when the package version is different between the server and your local environment
The version of package numpy in the local environment is 1.24.2, which does not fit the criteria for the requirement numpy. Your UDF might not work when the package version is different between the server and your local environment
The version of pack

In [15]:
#we are creating a stage called ml_models that can be used to store machine learning models that we will train 
session.sql('CREATE OR REPLACE STAGE ml_models ').collect()

[Row(status='Stage area ML_MODELS successfully created.')]

# Model Building

In [ ]:
#Necessary Library
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
import joblib
import os
import snowflake

#Model
def train_model(session: snowflake.snowpark.Session) -> float:
    snowdf = session.table("feature_store")
    #Dropping unwanted columns
    snowdf = snowdf.drop(['CUSTOMER_SK', 'C_CURRENT_HDEMO_SK', 'C_CURRENT_ADDR_SK', 'C_CUSTOMER_ID', 'CA_ADDRESS_SK', 'CD_DEMO_SK'])
    snowdf_train, snowdf_test = snowdf.random_split([0.8, 0.2], seed=82) 

    # save the train and test sets as time stamped tables in Snowflake 
    snowdf_train.write.mode("overwrite").save_as_table("tpcds_xgboost.demo.tpc_TRAIN")
    snowdf_test.write.mode("overwrite").save_as_table("tpcds_xgboost.demo.tpc_TEST")
    train_x = snowdf_train.drop("TOTAL_SALES").to_pandas() # drop labels for training set
    train_y = snowdf_train.select("TOTAL_SALES").to_pandas()
    test_x = snowdf_test.drop("TOTAL_SALES").to_pandas()
    test_y = snowdf_test.select("TOTAL_SALES").to_pandas()
    cat_cols = ['CA_ZIP', 'CD_GENDER', 'CD_MARITAL_STATUS', 'CD_CREDIT_RATING', 'CD_EDUCATION_STATUS']
    num_cols = ['C_BIRTH_YEAR', 'CD_DEP_COUNT']

    #Transformer to handle numerical values
    num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])

    #Processing numerical and categorical columns
    preprocessor = ColumnTransformer(
    transformers=[('num', num_pipeline, num_cols),('encoder', OneHotEncoder(handle_unknown="ignore"), cat_cols) ])

    #pipline for dataprocessinga and model fitting
    pipe = Pipeline([('preprocessor', preprocessor), ('xgboost', XGBRegressor())])
    pipe.fit(train_x, train_y)

    #model evaluation
    test_preds = pipe.predict(test_x)
    rmse = mean_squared_error(test_y, test_preds)
    mape = mean_absolute_percentage_error(test_y, test_preds)

    
    #storing trained model into snowflake staging area
    model_file = os.path.join('/tmp', 'model.joblib')
    joblib.dump(pipe, model_file)
    session.file.put(model_file, "@ml_models",overwrite=True)
    return mape



In [16]:
#Necessary Library
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import joblib
import os
import snowflake

#Model
def train_model(session: snowflake.snowpark.Session) -> float:
    snowdf = session.table("feature_store")
    #Dropping unwanted columns
    snowdf = snowdf.drop(['CUSTOMER_SK', 'C_CURRENT_HDEMO_SK', 'C_CURRENT_ADDR_SK', 'C_CUSTOMER_ID', 'CA_ADDRESS_SK', 'CD_DEMO_SK'])
    snowdf_train, snowdf_test = snowdf.random_split([0.8, 0.2], seed=82) 

    # save the train and test sets as time stamped tables in Snowflake 
    snowdf_train.write.mode("overwrite").save_as_table("tpcds_xgboost.demo.tpc_TRAIN")
    snowdf_test.write.mode("overwrite").save_as_table("tpcds_xgboost.demo.tpc_TEST")
    train_x = snowdf_train.drop("TOTAL_SALES").to_pandas() # drop labels for training set
    train_y = snowdf_train.select("TOTAL_SALES").to_pandas()
    test_x = snowdf_test.drop("TOTAL_SALES").to_pandas()
    test_y = snowdf_test.select("TOTAL_SALES").to_pandas()
    cat_cols = ['CA_ZIP', 'CD_GENDER', 'CD_MARITAL_STATUS', 'CD_CREDIT_RATING', 'CD_EDUCATION_STATUS']
    num_cols = ['C_BIRTH_YEAR', 'CD_DEP_COUNT']

    #Transformer to handle numerical values
    num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])

    #Processing numerical and categorical columns
    preprocessor = ColumnTransformer(
    transformers=[('num', num_pipeline, num_cols),('encoder', OneHotEncoder(handle_unknown="ignore"), cat_cols) ])

    #pipline for dataprocessinga and model fitting
    pipe = Pipeline([('preprocessor', preprocessor), ('linear', LinearRegression())])
    pipe.fit(train_x, train_y)

    #model evaluation
    test_preds = pipe.predict(test_x)
    # rmse = mean_squared_error(test_y, test_preds)
    mape = mean_absolute_percentage_error(test_y, test_preds)

    
    #storing trained model into snowflake staging area
    model_file = os.path.join('/tmp', 'model.joblib')
    joblib.dump(pipe, model_file)
    session.file.put(model_file, "@ml_models",overwrite=True)
    return mape


In [18]:
#Training Model
session.use_warehouse('snowpark_opt_wh')
train_model_sp = F.sproc(train_model, session=session, replace=True, is_permanent=True, name="xgboost_sproc", stage_location="@ml_models")
# Switch to Snowpark Optimized Warehouse for training and to run the stored proc
train_model_sp(session=session)


0.7979973908907877

In [19]:
# Switch back to feature engineering/inference warehouse
session.use_warehouse('FE_AND_INFERENCE_WH')

# Prediction Engine

In [20]:
import sys
import pandas as pd
import cachetools
import joblib
from snowflake.snowpark import types as T

session.add_import("@ml_models/model.joblib")  

features = [ 'C_BIRTH_YEAR', 'CA_ZIP', 'CD_GENDER', 'CD_MARITAL_STATUS', 'CD_CREDIT_RATING', 'CD_EDUCATION_STATUS', 'CD_DEP_COUNT']

@cachetools.cached(cache={})
def read_file(filename):
       import_dir = sys._xoptions.get("snowflake_import_directory")
       if import_dir:
              with open(os.path.join(import_dir, filename), 'rb') as file:
                     m = joblib.load(file)
                     return m

@F.pandas_udf(session=session, max_batch_size=10000, is_permanent=True, stage_location='@ml_models', replace=True, name="clv_xgboost_udf")
def predict(df:  T.PandasDataFrame[int, str, str, str, str, str, int]) -> T.PandasSeries[float]:
       m = read_file('model.joblib')       
       df.columns = features
       return m.predict(df)

In [21]:
inference_df = session.table('feature_store')
inference_df = inference_df.drop(['CUSTOMER_SK', 'C_CURRENT_HDEMO_SK', 'C_CURRENT_ADDR_SK', 'C_CUSTOMER_ID', 'CA_ADDRESS_SK', 'CD_DEMO_SK'])
inputs = inference_df.drop("TOTAL_SALES")

snowdf_results = inference_df.select(*inputs,
                    predict(*inputs).alias('PREDICTION'), 
                    (F.col('TOTAL_SALES')).alias('ACTUAL_SALES'))

snowdf_results.write.mode('overwrite').save_as_table('predictions')